решить проблему с повторным показом изображения, сделать страницу ожидания и выход. дизайн

In [1]:
import os
import sys
from PyQt5.QtCore import Qt
from PyQt5.uic import loadUi
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QDialog, QApplication, QWidget, QMainWindow
from PyQt5.QtGui import QPixmap, QPixmapCache

class WelcomeScreen(QMainWindow):
    def __init__(self):
        super(WelcomeScreen,self).__init__()
        loadUi("page1.ui",self)
        pixmap = QPixmap('logo(2).png')
        self.label.setPixmap(pixmap)
        self.start_btn.clicked.connect(lambda: self.start_click())
    
    def start_click(self):
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class SelectScreen(QMainWindow):
    def __init__(self):
        super(SelectScreen,self).__init__()
        loadUi("page2.ui",self)
        self.setAcceptDrops(True)
        self.pushFolderButton.clicked.connect(lambda: self.open_folder())
        self.pushImageButton.clicked.connect(lambda: self.open_image())
    
    def open_folder(self):
        file = QtWidgets.QFileDialog.getExistingDirectory(self, 'Select Folder')
        if os.path.exists(file):
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def open_image(self):
        file = QtWidgets.QFileDialog.getOpenFileName(self, "Select Image", None, "Image (*.png *.jpg)")[0]
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def dragEnterEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dragMoveEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasImage:
            event.setDropAction(Qt.CopyAction)
            self.file_path = event.mimeData().urls()[0].toLocalFile()
            self.set_image(self.file_path)
            event.accept()
        else:
            event.ignore()

    def set_image(self, file):
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        else:
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)

class ImagePretrainScreen(QMainWindow):
    def __init__(self, file):
        super(ImagePretrainScreen,self).__init__()
        loadUi("page3_image.ui",self)
        pixmap = QPixmap(file)
        pixmap= pixmap.scaledToWidth(350)
        self.label.setPixmap(pixmap)
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops())
        self.ImageProcessingButton.clicked.connect(lambda: self.ImageProcessing())
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self):
        print("to do")
    def ImageProcessing(self):
        print("to do")
        
    def Back(self):
        self.label.clear()
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()-1)

class FolderPretrainScreen(QMainWindow):
    def __init__(self, file):
        super(FolderPretrainScreen,self).__init__()
        loadUi("page3_folder.ui",self)
        self.label.setText("Your choice: \n" + file)
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops())
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self):
        print("to do")
        
    def Back(self):
        self.label.clear()
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()-1)

# main
app = QApplication(sys.argv)
welcome = WelcomeScreen()
widget = QtWidgets.QStackedWidget()
widget.addWidget(welcome)
widget.setFixedHeight(590)
widget.setFixedWidth(800)
widget.show()
try:
    sys.exit(app.exec_())
except:
    print("Exiting")

C:/Users/USER/Downloads/drop/.ipynb_checkpoints
C:/Users/USER/Downloads/drop/data
Exiting
